In [ ]:
#!rm -rf runs
#!rm -rf frame
#!rm *

In [ ]:
!pip install ultralytics

In [ ]:
import os
import shutil
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
#from PIL import Image, ImageDraw
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import animation, rc
rc('animation', html='jshtml')
from IPython.display import Image, Video
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [ ]:
image_dir = '/kaggle/input/cars-video-object-tracking/images'
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.PNG')]
image_files = sorted(image_files)
images = []
for path in image_files:
    image=plt.imread(path)
    image=cv2.resize(image, dsize=None, fx=0.3, fy=0.3, interpolation=cv2.INTER_LINEAR)
    #print(image.shape)
    images+=[image]

def update(frame):
    plt.clf()
    plt.imshow(images[frame])
    plt.axis('off')

ani = FuncAnimation(plt.gcf(), update, frames=len(images), interval=300)
ani.save('detections.gif', writer='pillow', fps=30)

In [ ]:
Image(open('./detections.gif','rb').read())

---

In [ ]:
!mkdir yolov8_tracking
!mkdir yolov8_tracking/examples
!mkdir yolov8_tracking/examples/weights

In [ ]:
dir0='/kaggle/input/cars-video-object-tracking/images'

In [ ]:
!yolo mode=track tracker="bytetrack.yaml" model=yolov8n.pt conf=0.01 source={dir0} save=True save_txt=True

In [ ]:
image_dir2 = '/kaggle/working/runs/detect/predict'
image_files2 = [os.path.join(image_dir2, f) for f in os.listdir(image_dir2) if f.endswith('.PNG')]
image_files2 = sorted(image_files2)
images2 = [plt.imread(image_file) for image_file in image_files2]

def update2(frame):
    plt.clf()
    plt.imshow(images2[frame])
    plt.axis('off')

ani2 = FuncAnimation(plt.gcf(), update2, frames=len(images2), interval=300)
ani2.save('detections2.gif', writer='pillow', fps=30)

In [ ]:
Image(open('./detections2.gif','rb').read())

In [ ]:
!ls /kaggle/working/yolov8_tracking/runs/detect/predict

# Analyse Result of Tracking

In [ ]:
txt_dir='/kaggle/working/runs/detect/predict/labels'
txtpaths=[]
texts=os.listdir(txt_dir)
for item in texts:
    txtpaths+=[os.path.join(txt_dir,item)]

In [ ]:
boxdata=[]
boxfile=[]
for i in range(len(txtpaths)):
    file=txtpaths[i]
    boxdata+=[np.loadtxt(file)] 
    filename=file.split('/')[-1]
    if filename=='output.txt':
        filename='output_0.txt'
    boxfile+=[filename[0:-4]]


In [ ]:
BOX=pd.DataFrame(columns=range(7))

for i in range(len(boxdata)):
    if type(boxdata[i][0])==np.float64:
        add=pd.DataFrame([boxdata[i]])
        add[6]=boxfile[i]
        BOX=pd.concat([BOX,add],axis=0)
    else:
        add=pd.DataFrame(boxdata[i])
        add[6]=boxfile[i]
     
        BOX=pd.concat([BOX,add],axis=0)       

BOX2=BOX.iloc[:,0:7][BOX[0]!=0]#person
BOX2[0:5]

In [ ]:
BOX2.columns=['class','x','y','w','h','id','file']
BOX2['order']=BOX2['file'].apply(lambda x:x.split('_')[-1].zfill(3))
data0=BOX2.sort_values('order',ascending=True).reset_index(drop=True)
display(data0)
data0=data0.fillna(0)

In [ ]:
ids=sorted(data0['id'].unique().tolist())
orders=sorted(data0['order'].unique().tolist())
orders=pd.DataFrame(columns=['order'],data=orders)

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
for i in range(len(ids)):
    idi=ids[i]
    datai=data0[data0['id']==idi]
    datai=orders.merge(datai,on='order',how='left')
    idi2='id:'+str(int(idi))
    fig.add_trace(go.Scatter(x=datai['x'],y=-datai['y'],name=idi2,hovertext=idi2),secondary_y=False)
fig.update_layout(autosize=False,width=700,height=500,title_text="Car Tracking")
fig.update_xaxes(title_text="x")
fig.update_yaxes(title_text="y",secondary_y=False)
fig.show()

# Draw Tracking Line

In [ ]:
!mkdir /kaggle/working/frame2

In [ ]:
paths=[]
for dirname, _, filenames in os.walk('/kaggle/input/cars-video-object-tracking/images'): 
    for filename in filenames:
        if filename[-4:]=='.PNG':
            paths+=[(os.path.join(dirname, filename))]
paths=sorted(paths)

In [ ]:
data0

In [ ]:
colors14=[
    (255, 0, 0), (255, 69, 0), (255, 140, 0), (255, 165, 0), (255, 215, 0), 
    (255, 255, 0), (173, 255, 47), (127, 255, 0), (0, 255, 0), (0, 250, 154), 
    (0, 255, 255), (0, 191, 255), (0, 0, 255), (138, 43, 226)
    ]

## set cv2.circle

In [ ]:
images3=[]
for i,path in enumerate(paths):
    order=path.split('/')[-1].split('.')[0].split('_')[-1]

    img = plt.imread(path)
    H,W=img.shape[0],img.shape[1]
    orderint=int(order)
    
    for k in range(orderint+1):
        datai=data0[data0['order']==str(k).zfill(6)]
        for j in range(len(datai)):
            c=colors14[int(datai.iloc[j,5])%14]
            x,y=datai.iloc[j,1],datai.iloc[j,2]
            cv2.circle(img,(int(x*W),int(y*H)),4,(c),-1) #(img, center, radius, color, thickness)    
    images3+=[img]
    #cv2.imwrite('/kaggle/working/frame2/'+str(order)+'.PNG',img) 


In [ ]:
def update3(frame):
    plt.clf()
    plt.imshow(images3[frame])
    plt.axis('off')

ani3 = FuncAnimation(plt.gcf(), update3, frames=len(images3), interval=300)
ani3.save('detections3.gif', writer='pillow', fps=30)

In [ ]:
Image(open('./detections3.gif','rb').read())